<a href="https://colab.research.google.com/github/ZacharyZekaiXu/ZekaiXu_CrossmodalRecognition/blob/main/DDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>DDQN </h1>


In [9]:
!conda --version

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
/bin/bash: conda: command not found


In [7]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached meshplot-0.3.3-py3-none-any.whl
  Using cached jupyterlab-3.4.6-py3-none-any.whl (8.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 24.9 MB/s eta 0:00:00
  Using cached pybullet-3.2.5-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (91.7 MB)
  Using cached pyrender-0.1.45-py3-none-any.whl (1.2 MB)
  Using cached pythreejs-2.4.1-py3-none-any.whl (3.3 MB)
  Using cached Rtree-1.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 96.1 MB/s eta 0:00:00
  Using cached trimesh-3.14.1-py3-none-any.whl (660 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached jupyterlab_server-2.15.1-py3-none-any.whl (54 k

In [6]:
!bash download_models.sh

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
bash: download_models.sh: Transport endpoint is not connected


In [5]:
!bash download_data.sh

--2022-09-06 23:30:32--  https://dl.fbaipublicfiles.com/ABC/ActiveDataset.tar
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7539645424 (7.0G) [application/gzip]
Saving to: ‘ActiveDataset.tar.2’

ActiveDataset.tar.2 100%[===================>]   7.02G  22.4MB/s    in 5m 17s  

2022-09-06 23:35:50 (22.7 MB/s) - ‘ActiveDataset.tar.2’ saved [7539645424/7539645424]


gzip: stdin: Software caused connection abort
gzip: error closing file: Transport endpoint is not connected
tar: images_colourful/18062.npy: Cannot write: Software caused connection abort
tar: images_colourful/18062.npy: Cannot utime: Transport endpoint is not connected
tar: images_colourful/18062.npy: Cannot change ownership to uid 1185301026, gid 1185301026: Transport endpoint is not connected
tar: images_colourful/18062.n

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
%cd gdrive/MyDrive/Active-3D-Vision-and-Touch/

/content/gdrive/MyDrive/Active-3D-Vision-and-Touch


<h2> 1. DDQN - Training  </h2>

In [2]:
!pip install submitit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached submitit-1.4.5-py3-none-any.whl (73 kB)


In [3]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 41.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.3 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61263 sha256=06ee2638e22b8440b8e3132eea54ce1bd85a081e2f02f04ba2f344fb52262b44
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=4bcf9f7a208c7965dfa0a

Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py37_cu113_pyt1121/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 MB 29.4 MB/s eta 0:00:00


In [8]:
!pip install torch
import torch
!pip install trimesh
!pip install pyrender

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Here we train a DDQN model in the touch only setting with grasping. This model leverages the latent space to learn actions.

In [9]:
!pip3 install pybullet --upgrade
!pip3 install stable_baselines --upgrade
!pip3 install tensorflow==1.14.0 #tensorflow deprecates and breaks their API
#see PyBullet Quickstart Guide at
#https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit
import imageio
import numpy as np



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.5/734.5 kB 40.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.1 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.25.2-py3-none-any.whl size=852301 sha256=d4922b3ddb52771838cddfe5ae5c62b6bc831f1d70c9c0e31b822c2732566124
  Stored in directory: /root/.cache/p

In [10]:

!pip uninstall tensorboard
!pip uninstall tensorflow
!pip uninstall tensorboard
!pip uninstall tensorboardx
!pip uninstall tensorboard-plugin-wit
!pip uninstall cloud-tpu-client

!pip install tensorboard


Found existing installation: tensorboard 1.14.0
Uninstalling tensorboard-1.14.0:
  Would remove:
    /usr/local/bin/tensorboard
    /usr/local/lib/python3.7/site-packages/tensorboard-1.14.0.dist-info/*
    /usr/local/lib/python3.7/site-packages/tensorboard/*
Proceed (Y/n)? y
  Successfully uninstalled tensorboard-1.14.0
Found existing installation: tensorflow 1.14.0
Uninstalling tensorflow-1.14.0:
  Would remove:
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/site-packages/tensorflow-1.14.0.dist-info/*
    /usr/local/lib/python3.7/site-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-1.14.0
Found existing installation: tensorboard-plugin-wit 1.8.1
Uninstalling tensorboard-plugin-wit-1.8.1:
  Would remove:
    /usr/local/lib/python3.7/site-packages/tensorboard_plugin_wit-1.8.1.dist-info

In [11]:
from tensorboard import version
print(version.VERSION)

2.10.0


In [12]:
import os

from pterotactyl.policies.DDQN import train as ddqn_train

class Params: # define training arguments 
    def __init__(self):
        self.cut = 0.33
        self.seed = 0 
        self.gamma = .99
        self.limit_data = True
        self.epochs = 30
        self.lr = 0.0003
        self.eval = False
        self.env_batch_size = 2
        self.num_actions = 50
        self.budget = 5
        self.train_batch_size = 16
        self.epsilon_start = 1.
        self.epsilon_end = 0.001
        self.epsilon_decay = 0.9999
        self.number_points = 10000
        self.loss_coeff = 9000
        self.train_steps = 20
        self.valid_steps = 20 
        self.target_update = 1000
        self.burn_in = 100
        self.use_latent = True
        self.use_recon = False 
        self.exp_id = "example"
        self.exp_type = "ddqn"
        self.finger = False 
        self.num_grasps = 5 
        self.mem_capacity = 1000
        self.hidden_dim = 100
        self.layers = 5
        self.use_touch = True # using touch 
        self.use_img = False # not using vision 
        self.normalization = 'first'
        self.visualize = False
        self.pretrained_recon = True
        self.pretrained=False
    

params = Params()
ddqn_trainer = ddqn_train.Engine(params)


In [13]:
ddqn_train.Engine(params)

In [14]:
ddqn_trainer()

/usr/local/lib/python3.7/site-packages/pytorch3d/io/obj_io.py:529: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


FileNotFoundError: ignored

<h2> 2. DDQN - Evaluation </h2>

Here we evaluate the trained DDQN model in the touch only grasping setting. 

In [15]:
import os
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
from glob import glob 

from pterotactyl import objects

FONT_LOCATION = os.path.dirname(objects.__file__) + '/OpenSans-Bold.ttf'


params.eval = True 
params.visualize = True
tester = ddqn_train.Engine(params)
tester()

histogram = Image.open('results/ddqn/example/histogram.png')
display(histogram)
print('\n\n')
actions = Image.open('results/ddqn/example/sphere_projection.png')
print('   Projected Actions')
display(actions)

img_locations = glob('results/ddqn/example/*/')
for location in img_locations[:5]:
    mesh = Image.open(location + "mesh.png")
    points = Image.open(location + "points.png")
    gt = Image.open(location + "ground_truth.png")
        
    combo = Image.new('RGB', (512*3, 512))
    x_offset = 0
    
    for im, st in [[gt, 'Ground Truth'], [mesh, 'Mesh'], [points, 'Point Cloud']]:
        
        draw = ImageDraw.Draw(im)
        font = ImageFont.truetype(FONT_LOCATION, 35)
        W,H = im.size
        w,h = font.getsize(st)
        draw.text(((W-w)/2, 0),st,(0, 0, 0), font = font)
        combo.paste(im, (x_offset, 0))
        x_offset += im.size[0]
        
    display(combo)


FileNotFoundError: ignored

<h2> DDQN - Pretrained </h2>

We now evaluate the pretrained policy: 

In [ ]:
import os

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
from glob import glob 

from pterotactyl import objects
from pterotactyl.policies.DDQN import train as ddqn_train 

FONT_LOCATION = os.path.dirname(objects.__file__) + '/OpenSans-Bold.ttf'


class Params: # define training arguments 
    def __init__(self):
        self.cut = 0.33
        self.seed = 0 
        self.gamma = .99
        self.limit_data = True
        self.epochs = 30
        self.lr = 0.0003
        self.eval = True
        self.env_batch_size = 2
        self.num_actions = 50
        self.budget = 5
        self.train_batch_size = 16
        self.epsilon_start = 1.
        self.epsilon_end = 0.001
        self.epsilon_decay = 0.9999
        self.number_points = 10000
        self.loss_coeff = 9000
        self.train_steps = 20
        self.valid_steps = 20 
        self.target_update = 1000
        self.burn_in = 100
        self.use_latent = False
        self.use_recon = True 
        self.exp_id = "ddqn_pre"
        self.exp_type = "ddqn"
        self.finger = False 
        self.num_grasps = 5 
        self.mem_capacity = 1000
        self.hidden_dim = 100
        self.layers = 5
        self.use_touch = True # using touch 
        self.use_img = False # not using vision 
        self.normalization = 'first'
        self.visualize = True
        self.pretrained_recon = True
        self.pretrained = True 
    

params = Params()
eval = ddqn_train.Engine(params)
eval()


histogram = Image.open('results/ddqn/ddqn_pre/histogram.png')
display(histogram)
print('\n\n')
actions = Image.open('results/ddqn/ddqn_pre/sphere_projection.png')
print('   Projected Actions')
display(actions)



img_locations = glob('results/ddqn/ddqn_pre/*/')
for location in img_locations[:5]:
    mesh = Image.open(location + "mesh.png")
    points = Image.open(location + "points.png")
    gt = Image.open(location + "ground_truth.png")
    
    
    combo = Image.new('RGB', (512*3, 512))
    x_offset = 0
    
    for im, st in [[gt, 'Ground Truth'], [mesh, 'Mesh'], [points, 'Point Cloud']]:
        
        draw = ImageDraw.Draw(im)
        font = ImageFont.truetype(FONT_LOCATION, 35)
        W,H = im.size
        w,h = font.getsize(st)
        draw.text(((W-w)/2, 0),st,(0, 0, 0), font = font)
        combo.paste(im, (x_offset, 0))
        x_offset += im.size[0]
        
    display(combo)






